## Parallelizing the computation, 2nd approach

In the last notebook we have created a table of track ids and offensiveness.
But we have lost the detailed multi-index structure.

The flattening is not necessary for pandas outer-join to work.
We repeat the procedure and generate a second table which has a deeply nested multiindex.

The structure of this notebook is basically identical to the previous one.

In [2]:
import numpy as np
import pandas as pd

In [3]:
word_table = pd.read_pickle("../pickles/word_table_cleaned.pickle")
word_table.head()

,category,strength,target
word,,,
bonk,non-discriminatory,mild,None
bukkake,non-discriminatory,strong,None
cocksucker,non-discriminatory,strong,None
dildo,non-discriminatory,strong,None
ho,non-discriminatory,strong,None


In [5]:
import sqlite3
conn = sqlite3.connect("../datasets/mxm_dataset.db")

cursor = conn.cursor()
cursor.execute("SELECT track_id, word, count FROM lyrics ORDER BY track_id;")
track_word_count = cursor.fetchall()
cursor.close()
track_word_count[:5]

[('TRAAAAV128F421A322', 'i', 6),
 ('TRAAAAV128F421A322', 'the', 4),
 ('TRAAAAV128F421A322', 'you', 2),
 ('TRAAAAV128F421A322', 'to', 2),
 ('TRAAAAV128F421A322', 'and', 5)]

In [6]:
sqldb_frame = pd.DataFrame(track_word_count, columns=["track_id", "word", "count"])
del track_word_count

In [7]:
sqldb_frame["word"]=sqldb_frame["word"].astype(str)

## Performing an outer joint to match words between lyrics and offensiveness rating

In [8]:
joint = sqldb_frame.join(word_table, on="word", how="left", lsuffix='_caller', rsuffix='_other')
print(joint.shape)
joint.head()

(19045332, 6)


,track_id,word,count,category,strength,target
0,TRAAAAV128F421A322,i,6,NaN,NaN,NaN
1,TRAAAAV128F421A322,the,4,NaN,NaN,NaN
2,TRAAAAV128F421A322,you,2,NaN,NaN,NaN
3,TRAAAAV128F421A322,to,2,NaN,NaN,NaN
4,TRAAAAV128F421A322,and,5,NaN,NaN,NaN


In [9]:
joint_indexed = joint.set_index(["track_id", "category", "strength", "target"])
joint_indexed.loc[("TRAADYI128E078FB38",),]

word  count
category           strength  target                      
NaN                NaN       NaN                 i     16
                             NaN               the     26
                             NaN               you      6
                             NaN                to     13
                             NaN               and     29
                             NaN                 a      2
                             NaN                me      3
                             NaN                it      7
                             NaN               not      5
                             NaN                in      9
                             NaN                my     29
                             NaN                is     10
                             NaN                of      1
                             NaN              your      3
                             NaN              that      6
                             NaN                do     23
                             NaN                on      7
                             NaN               are      2
                             NaN                we     29
                             NaN                am      3
                             NaN              will      4
                             NaN               all     38
                             NaN               for      1
                             NaN                no      1
                             NaN                be      1
                             NaN              have      1
                             NaN              love      2
                             NaN                so      1
                             NaN              know      2
                             NaN              this     40
...                                            ...    ...
                             NaN            walkin      1
                             NaN             south      1
                             NaN             pound      1
                             NaN              suit      1
                             NaN               ima      1
non-discriminatory strongest NaN        motherfuck      1
NaN                NaN       NaN           favorit      1
                             NaN               toe      1
                             NaN             class      1
                             NaN              spot      1
                             NaN                 w      3
                             NaN            playin      1
                             NaN            receiv      1
                             NaN              coat      1
                             NaN             cheek      7
                             NaN              whip      1
                             NaN               hop      1
                             NaN             chick      1
                             NaN             givin      1
                             NaN              mist      1
                             NaN              team      1
                             NaN              gear      1
                             NaN            repres      1
                             NaN            smokin      1
                             NaN               ref      1
                             NaN            stroll      1
                             NaN              butt      1
discriminatory     mild      sexuality      shorti      3
NaN                NaN       NaN              peep      1
                             NaN               boo      1

[197 rows x 2 columns]

## Aggregating the data
The joint table uses track ids and offensiveness categories as indices. This is what we want, but we still have individual cells for every word.

Now we aggregate the items for every index. We sum the entries. This gives us the total count of words in each category.

In [10]:
joint_indexed.index.is_unique

False

In [11]:
joint_indexed_filtered = joint_indexed["count"]
joint_indexed_filtered.head()

track_id            category  strength  target
TRAAAAV128F421A322  NaN       NaN       NaN       6
                                        NaN       4
                                        NaN       2
                                        NaN       2
                                        NaN       5
Name: count, dtype: int64

In [12]:
joint_aggregated = joint_indexed_filtered.agg("sum")

In [13]:
offensiveness_rating = joint_indexed_filtered.groupby(str, axis=0).agg("sum")

In [14]:
offensiveness_rating.head()

('TRAAAAV128F421A322', 'non-discriminatory', 'mild', nan)      1
('TRAAAAV128F421A322', nan, nan, nan)                        102
('TRAAABD128F429CF47', nan, nan, nan)                        226
('TRAAAED128E0783FAB', nan, nan, nan)                        421
('TRAAAEF128F4273421', nan, nan, nan)                        139
Name: count, dtype: int64

In [15]:
offensiveness_rating.to_pickle("../pickles/offensiveness_rating_structured")